# Deploy trained Keras or TensorFlow models using Amazon SageMaker
Amazon SageMaker makes it easier for any developer or data scientist to build, train, and deploy machine learning (ML) models. While it’s designed to alleviate the undifferentiated heavy lifting from the full life cycle of ML models, Amazon SageMaker’s capabilities can also be used independently of one another; that is, models trained in Amazon SageMaker can be optimized and deployed outside of Amazon SageMaker (or even out of the cloud on mobile or IoT devices at the edge). Conversely, Amazon SageMaker can deploy and host pre-trained models from model zoos, or other members of your team.

Amazon SageMaker deploy your model to a TensorFlow Serving-based server. The server provides a super-set of the [TensorFlow Serving REST API](https://www.tensorflow.org/serving/api_rest).

To Deploy a Keras/TensorFlow Model, You'll need to save your model in a TensorFlow SavedModel format (Already implemented in the script).  

for More Info take a look at the ```def save_model(model, output):``` in the script

## Deploy a trained Model
Instead of retraining the model,  we'll start from one of the previously trained models.

In [ ]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()

Configure the estimator as it was configured in notebook:  1 - Monitoring your TensorFlow scripts

In [ ]:
from sagemaker.tensorflow import TensorFlow
estimator = ...

In [ ]:
estimator = estimator.attach(training_job_name=)  ## Configure with your previous cifar10 job name (First Completed job. You can find that name from the SageMaker console

In [ ]:
predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

## Make some predictions
To verify the that the endpoint functions properly, we generate random data in the correct shape and get a prediction.

In [ ]:
# Creating fake prediction data
import numpy as np
data = np.random.randn(1, 32, 32, 3)
print("Predicted class is {}".format(np.argmax(predictor.predict(data)['predictions'])))

### Calculating accuracy and create a confusion matrix based on the test dataset

Our endpoint works as expected, we'll now use the test dataset for predictions and calculate our model accuracy

In [ ]:
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
datagen = ImageDataGenerator()

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

def predict(data):
    predictions = predictor.predict(data)['predictions']
    return predictions

In [ ]:
batch_size = 128
predicted = []
actual = []
batches = 0
for data in datagen.flow(x_test,y_test,batch_size=batch_size):
    for i,prediction in enumerate(predict(data[0])):
        predicted.append(np.argmax(prediction))
        actual.append(data[1][i][0])
    batches += 1
    if batches >= len(x_test) / batch_size:
        break

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

accuracy = accuracy_score(y_pred=predicted,y_true=actual)
display('Average accuracy: {}%'.format(round(accuracy*100,2)))

In [ ]:
%matplotlib inline
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

cm = confusion_matrix(y_pred=predicted,y_true=actual)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
sn.set(rc={'figure.figsize':(11.7,8.27)})
sn.set(font_scale=1.4)#for label size
sn.heatmap(cm, annot=True,annot_kws={"size": 10})# font size

Using this heatmap we can calculate the accuracy of each one of the labels

# Cleaning up
To avoid incurring charges to your AWS account for the resources used in this tutorial you need to delete the SageMaker Endpoint:

In [ ]:
sagemaker_session.delete_endpoint(predictor.endpoint)